In [84]:
import pandas as pd
import numpy as np

In [85]:
import os

def get_csvs(dir:str):
    dirList=os.listdir(dir)
    endResult=[]
    for dire in dirList:
        if(os.path.join(dir,dire).endswith(".csv")):
            endResult.append(dir+"/"+dire)
    return endResult 

In [86]:
allDirs=get_csvs("ScrapedData")

In [105]:
def irregularitiesFinder(df:pd.DataFrame, stockName:str, dic:dict):
    # Calculate percentage change
    df['Pct_Change'] = df['Close'].pct_change()

    # Flag large changes
    threshold = 0.13  # Define a threshold for irregular change, e.g., 10%
    irregularities = df.index[abs(df['Pct_Change']) > threshold].to_list();


    from datetime import timedelta
    for i, index in enumerate(irregularities):
        if(pd.to_datetime(df.iloc[index,0])-pd.to_datetime(df.iloc[index-1,0])>timedelta(days=3)):
            irregularities.remove(index)

    if(len(irregularities)==0):
        return

    irregularitiesNP=np.zeros((len(irregularities),10,7),dtype=object);
    for i, index in enumerate(irregularities):

        irregularitiesNP[i,:,:]=df.iloc[index-5:index+5,:].to_numpy()

    dic[stockName]=irregularitiesNP

In [2]:
df=pd.read_csv("ScrapedData/CCOLA.IS.csv")

,Date,Open,High,Low,Close,Volume
0,2019-10-22 00:00:00+03:00,31.259470,31.376331,30.967327,31.064709,344403.666667
1,2019-10-23 00:00:00+03:00,31.181570,31.415283,30.986805,31.201046,324291.000000
2,2019-10-24 00:00:00+03:00,31.337378,32.135906,31.337378,32.077477,273870.200000
3,2019-10-25 00:00:00+03:00,32.135910,32.174862,31.493190,31.629524,302192.600000
4,2019-10-28 00:00:00+03:00,32.019050,32.116430,31.746378,31.746378,312042.200000


In [106]:
stDic={}
for dirs in allDirs:
    df=pd.read_csv(dirs)

    irregularitiesFinder(df,dirs,stDic)
stDic.keys()

dict_keys(['ScrapedData/ALARK.IS.csv', 'ScrapedData/BRSAN.IS.csv', 'ScrapedData/CCOLA.IS.csv', 'ScrapedData/DOAS.IS.csv', 'ScrapedData/HALKB.IS.csv', 'ScrapedData/KONTR.IS.csv', 'ScrapedData/ODAS.IS.csv', 'ScrapedData/PGSUS.IS.csv', 'ScrapedData/THYAO.IS.csv', 'ScrapedData/VAKBN.IS.csv', 'ScrapedData/VESTL.IS.csv', 'ScrapedData/XBANK.IS.csv', 'ScrapedData/XU030.IS.csv', 'ScrapedData/XU100.IS.csv', 'ScrapedData/XUSIN.IS.csv', 'ScrapedData/ZOREN.IS.csv'])

In [109]:
len(stDic)

16

In [110]:
for stockName in stDic:
    print(stockName)
    print(stDic[stockName].shape)
        

ScrapedData/ALARK.IS.csv
(1, 10, 7)
ScrapedData/BRSAN.IS.csv
(3, 10, 7)
ScrapedData/CCOLA.IS.csv
(1, 10, 7)
ScrapedData/DOAS.IS.csv
(2, 10, 7)
ScrapedData/HALKB.IS.csv
(1, 10, 7)
ScrapedData/KONTR.IS.csv
(1, 10, 7)
ScrapedData/ODAS.IS.csv
(2, 10, 7)
ScrapedData/PGSUS.IS.csv
(1, 10, 7)
ScrapedData/THYAO.IS.csv
(1, 10, 7)
ScrapedData/VAKBN.IS.csv
(1, 10, 7)
ScrapedData/VESTL.IS.csv
(1, 10, 7)
ScrapedData/XBANK.IS.csv
(1, 10, 7)
ScrapedData/XU030.IS.csv
(1, 10, 7)
ScrapedData/XU100.IS.csv
(1, 10, 7)
ScrapedData/XUSIN.IS.csv
(1, 10, 7)
ScrapedData/ZOREN.IS.csv
(1, 10, 7)


In [111]:
stDic["ScrapedData/XU030.IS.csv"]

array([[['2020-07-20 00:00:00+03:00', 134319.0, 135191.2, 133921.5,
         134530.0, 2371520.0, 0.001122197482943621],
        ['2020-07-21 00:00:00+03:00', 135423.1, 136198.4, 132451.3,
         133454.9, 2289060.0, -0.007991526053668374],
        ['2020-07-22 00:00:00+03:00', 134241.3, 134877.6, 133216.7,
         133703.4, 2388580.0, 0.0018620522738392875],
        ['2020-07-23 00:00:00+03:00', 134385.3, 135546.8, 134194.0,
         134821.2, 2334200.0, 0.008360295998456335],
        ['2020-07-24 00:00:00+03:00', 134220.8, 134947.3, 133890.9,
         134849.0, 2294480.0, 0.00020619902507901244],
        ['2020-07-27 00:00:00+03:00', 1356.1, 1361.3, 1347.5, 1350.0,
         2085780.0, -0.9899888022899688],
        ['2020-07-28 00:00:00+03:00', 1350.0, 1351.8, 1301.0, 1301.0,
         1898120.0, -0.03629629629629627],
        ['2020-07-29 00:00:00+03:00', 1304.3, 1307.2, 1258.8, 1272.2,
         2033940.0, -0.022136817832436595],
        ['2020-07-30 00:00:00+03:00', 1273.0, 1273.5

In [112]:
mistakesDf = pd.DataFrame(stDic["ScrapedData/XU030.IS.csv"][0], columns=df.columns)

In [113]:
mistakesDf

,Date,Open,High,Low,Close,Volume,Pct_Change
0,2020-07-20 00:00:00+03:00,134319.0,135191.2,133921.5,134530.0,2371520.0,0.001122
1,2020-07-21 00:00:00+03:00,135423.1,136198.4,132451.3,133454.9,2289060.0,-0.007992
2,2020-07-22 00:00:00+03:00,134241.3,134877.6,133216.7,133703.4,2388580.0,0.001862
3,2020-07-23 00:00:00+03:00,134385.3,135546.8,134194.0,134821.2,2334200.0,0.00836
4,2020-07-24 00:00:00+03:00,134220.8,134947.3,133890.9,134849.0,2294480.0,0.000206
5,2020-07-27 00:00:00+03:00,1356.1,1361.3,1347.5,1350.0,2085780.0,-0.989989
6,2020-07-28 00:00:00+03:00,1350.0,1351.8,1301.0,1301.0,1898120.0,-0.036296
7,2020-07-29 00:00:00+03:00,1304.3,1307.2,1258.8,1272.2,2033940.0,-0.022137
8,2020-07-30 00:00:00+03:00,1273.0,1273.5,1257.9,1259.6,2353220.0,-0.009904
9,2020-08-04 00:00:00+03:00,1273.4,1276.5,1199.7,1221.3,2567860.0,-0.030406
